### Programming for Science and Finance

*Prof. Götz Pfeiffer, School of Mathematical and Statistical Sciences, University of Galway*



# Notebook 9: Programming for Finance

This notebook accompanies **Part III**. You will:

* connect **compound interest** with the differential equation $A' = rA$
* use **Euler’s method** to approximate solutions and interpret discrete compounding as an ODE solver
* apply **root-finding** techniques to determine a bond’s **yield to maturity**
* use the **Black–Scholes** model to compute theoretical option prices
* solve for an option’s **implied volatility** by finding roots of the pricing equation
* see how numerical ODEs and root solvers naturally support key ideas in financial mathematics

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt


## Task 1. Compound Interest as a Differential Equation

Many processes in finance grow proportionally to their current size:

* a bank account with continuously compounded interest
* population of investors or accounts
* simple models of price growth

This leads to one of the simplest and most important differential equations:
$$
\frac{dV}{dt} = r V(t),
$$
where:

* $V(t)$ = value of an account at time $t$
* $r$ = growth rate (interest rate)
Interpretation: The **instantaneous** growth rate is proportional to the account value.

The exact solution is the exponential:
$$
V(t) = V_0\,e^{rt}.
$$
aka “continuous compounding.”

* Euler's Method:  with $f(t, V) = r V(t)$,
  $$
  V_{n+1} = V_n + h r V_n = (1 + hr) V_n = (1 + hr)^{n+1} V_0
  $$

* If we take $n$ steps over $t$ years, then $h = t/n$ and $V_n = V_0 (1 + hr)^n$.
* As $n \to \infty$, we have $h \to 0$ and $(1 + hr)^n \to e^{rt}$.

Recall the code for the Euler Method:

In [ ]:
def euler(f, t0, y0, h):
    def approx(n):
        t, y = t0, y0
        for i in range(n):
            y += h * f(t, y)
            t += h
        return y
    return approx

The returned method `approx` computes one value of $y$ at a time (and repeats the computation of all previous values).  
It would be more efficient to grow the entire list of values incrementally.

In [ ]:
def euler_approx(f, t0, y0, h, n):
    approx = np.zeros(n+1)
    approx[0] = y0
    ti = t0
    for i in range(n):
        approx[i+1] = approx[i] + h * f(ti, approx[i])
        ti += h
    return approx

In fact, since $f(t, y)$ here is a linear function of $y$, the computation can be fully linearized, as follows (using `r` as parameter in place of the function `f`)

In [ ]:
def euler_growth(r, y0, h, n):
    return y0 * ((1 + h*r) ** np.arange(n+1))

In [ ]:
r = 0.035
t0, y0 = 0, 100
t, n = 15, 15 
h = t/n
f = lambda t, y: r * y
growth = euler(f, t0, y0, h)

This gives us three different ways to compute the same list of values using Euler’s method in different forms.  
We can compare them for speed with these commands:
```python
    %timeit g0 = np.array([growth(i) for i in range(n+1)])
    %timeit g1 = euler_approx(f, t0, y0, h, n)
    %timeit g2 = euler_growth(r, y0, h, n)
```

Let's check whether the computed values are identical:

In [ ]:
g0 = np.array([growth(i) for i in range(n+1)])
g1 = euler_approx(f, t0, y0, h, n)
g2 = euler_growth(r, y0, h, n)
print(g0, g1, g2, sep="\n")

And let's plot the approximate values against the exact solution, i.e., compound interest vs. continuous interest.

In [ ]:
# Parameters
t0, y0 = 0, 100
r = 0.135   # 13.5% interest
t, n = 10, 20
h = t/n

compound = euler_growth(r, y0, h, n)
times = np.linspace(0, t, n+1)
continuous = y0 * np.exp(r * times)

plt.plot(times, continuous, label="Exact e^{rt}")
plt.plot(times, compound, "o--", label="Euler Approx")
plt.xlabel("Time")
plt.ylabel("Value")
plt.legend()
plt.title("Compound Interest via Euler’s Method")
plt.show()


* Euler’s method slightly **underestimates** the true exponential (typical for this ODE).
* Discrete compounding with $m$ periods per year is simply Euler’s method with step size $h = 1/m$.

---
**Exercises.**

1. Compare approximations for $n = 10, 20, 50, 200$.

2. Compute the effective annual return for finite compounding:
  $(1 + hr)^{1/h} - 1$ and
  show that it converges to $e^r - 1$.

3. Show that the present value of a cashflow discounted continuously at rate $r$ is $e^{-rt}$.

4. Relate the discount factor to solving the ODE $y' = r y$ backwards in time.
---



## Task 2: Bond Pricing and Yield to Maturity (YTM)

Bonds are among the simplest financial instruments, yet they lead directly to a practical use of **numerical root finding**.  
A bond has:

* a face value $F$ (e.g., $1000)
* a coupon payment $C$ paid once per period
* $n$ payment periods
* a market price $P_{\text{market}}$

The **yield to maturity** $y$ is the single constant discount rate that makes the discounted value of all future payments equal the observed price.

There is **no closed-form formula** for $y$, so we must **solve a nonlinear equation numerically** by Root Finding.

### Bond Pricing Formula

If the yield is $y$, the theoretical price of a bond is:
$$
P(y) = \sum_{k=1}^{n} \frac{C}{(1+y)^k} + \frac{F}{(1+y)^n}.
$$
Given the market price $P_{\text{market}}$, the **YTM is the root** of
$$
f(y) = P(y) - P_{\text{market}} = 0.
$$

* As $y \to 0$, $P(y)$ is large (discounting is small).
* As $y \to \infty$, $P(y) \to 0$.
* The function is **smooth and strictly decreasing**.

This means that there is **exactly one root**.


**Example.**

* Face value: $F = 1000$
* Coupon: $C = 30$
* Periods: $n = 20$
* Market price: $P_{\text{market}} = 920$.

We solve $P(y) = 920$ for $y$.

Recall the `RootFinder` class.

In [ ]:
class RootFinder:
    def __init__(self, f, f1=None, eps=1e-6, M=99):
        self.f = f
        self.f1 = f1
        self.eps = eps
        self.M = M

    def solve(self, a, b= None):
        raise NotImplementedError("Subclass must implement solve().")

class BisectionSolver(RootFinder):
    def solve(self, a, b):  # overwrite parent's method
        f = self.f
        assert f(a) * f(b) < 0, "end points must have opposite signs"
        for i in range(self.M):
            c = (a+b)/2
            if abs(f(c)) < self.eps:
                return c
            if f(a) * f(c) < 0:
                b = c
            else:
                a = c
        print("Warning: maximum iterations reached.")
        return c  # now i >= M

class NewtonSolver(RootFinder):
    def solve(self, a, b=None):
        f, f1 = self.f, self.f1
        assert f1 is not None, "f1 needed"
        for i in range(self.M):
            y = f(a)
            if abs(y) < self.eps:
                return a
            a -= y/f1(a)
        print("Warning: maximum iterations reached.")
        return a  # now i >= M

Setup:

In [ ]:
C = 30    # Example:
F = 1000 # face value
n = 20   # number of payment periods
P = 920  # market price

f = lambda y: np.sum(C/ (1+y)**np.arange(1, n+1)) + F / (1+y)**n - P
solver = BisectionSolver(f, eps=1e-8)

ytm = solver.solve(0, 1)
print("Yield to Maturity:", ytm)

So the YTM is approximately 3.6%.  That is:

* The bond “acts like” a bank account that pays 3.6% per period if held to maturity.
* This number includes both coupon payments and the capital gain/loss at maturity.

It is a **unifying rate** summarizing the entire cash-flow structure.

---
**Exercises.**

1.  Compute the price for several yields $y$.  Plot $P(y)$.  Is it monotone?

2.  Compute the YTM for various market prices.  How does the YTM depend on the market price?

3.  Compute YTM for several real bond quotes from a financial website.

---

## Task 3 — Options and Implied Volatility

Options are financial contracts whose value depends on the price of an underlying asset.  
The most widely used theoretical pricing formula is the **Black–Scholes model**, which gives the price of a **European call option** as:
$$
C_{\text{BS}}(S, K, r, \sigma, T).
$$
Here:

* $S$ = current stock price
* $K$ = strike price
* $r$ = interest rate
* $\sigma$ = volatility of the stock
* $T$ = time to maturity

In real markets one can
observe the option’s **market price**.  
The **volatility** $\sigma$, however, is not known.  
But the market behaves **as if** the option has some volatility consistent with its price.  
This is called the **Implied Volatility** (IV).  
IV is the value of $\sigma$ that makes the Black–Scholes price equal the observed market price.  
There is **no closed-form formula** for $\sigma$.
**Root Finding** to the rescue ...


###  Black–Scholes Formula

For a European call option:
$$
C_{\text{BS}}(S, K, r, \sigma, T) = S\,N(d_1) - K e^{-rT} N(d_2),
$$
with
$$
d_1 = \frac{\ln(S/K) + (r + \tfrac{1}{2}\sigma^2)T}{\sigma\sqrt{T}},
\qquad
d_2 = \frac{\ln(S/K) + (r - \tfrac{1}{2}\sigma^2)T}{\sigma\sqrt{T}},
$$
i.e., $d_2 = d_1 - \sigma\sqrt{T}$.


Here $N(\cdot)$ is the [standard normal CDF](https://en.wikipedia.org/wiki/Normal_distribution) $\Phi(x)$, which in numpy can be computed with the help of the error function:
$$
\Phi(x) = \frac12 \left(1 + \mathrm{erf}\Bigl(\frac{x}{\sqrt{2}}\Bigr)  \right)
$$

In [ ]:
def phi(x):
    return 0.5 * (1 + math.erf(x / math.sqrt(2)))

###  Root Finding Problem

Given: $S, K, r, T$ and a market price $C$, solve
$$
f(\sigma) = C_{\text{BS}}(S, K, r, \sigma, T) - C = 0.
$$

Properties of $f$:

* $f(\sigma)$ is **smooth and increasing** in $\sigma$.
* As $\sigma \to 0$, $f(\sigma) \to \max(S-Ke^{-rT},0)-C$.
* As $\sigma \to \infty$, $f(\sigma) \to S - C$.
* Hence there is **exactly one root**.

For a Python implementation, we start with the Black-Scholes price:

In [ ]:
def BS_price(S, K, r, sigma, T):
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * phi(d1) - K * np.exp(-r*T) * phi(d2)

Then, we set up the parameters for a particular example.

In [ ]:
# Example value:
S = 100   # stock price
K = 100   # strike
r = 0.02  # 2% interest
T = 1.0   # 1 year
C = 10.0   # market price

From these data we can build the function $f(x)$, and the bisection solver for $f(x) = 0$.  
We choose an interval $[a, b]$ wide enough to safely contain the root.  
For typical options, $\sigma$ is rarely above $2$, or below $0.01$.

In [ ]:
a, b = 1e-6, 3.0
eps = 1e-8
f = lambda x : BS_price(S, K, r, x, T) - C
solver = BisectionSolver(f, eps)

... and finally run the solver with initial interval $[a, b]$.

In [ ]:
iv = solver.solve(a, b)
print("Implied Volatility:", iv)

The implied volatility is 23%.  
This means that the option’s market price is consistent with traders collectively assuming the stock has about 23% annual volatility.

---
**Exercises.**

1. Plot the Black–Scholes price as a function of volatility $\sigma$.
Is it increasing and smooth?

2. Compute or estimate the derivative of the function `f(x)` in the above example.
   Then use Newton's method to solve $f(x) = 0$ for $x$.  Compare your root $x$ to the one above.

3. Compute implied volatility for several other call options with different strikes.

---

## Summary

This notebook connects the numerical methods from previous notebooks with some core ideas from finance.
Specifically, we

* interpreted **compound interest** as the ODE (A' = rA) and related it to Euler’s method
* showed how **Euler updates** reproduce discrete compounding at different step sizes
* applied **root-finding** techniques to compute a bond’s **yield to maturity**
* used the **Black–Scholes formula** to model option prices
* solved for **implied volatility** by finding the root of the pricing equation
* illustrated how ODE solvers and numerical root methods appear naturally in financial computations

These examples illustrate how ordinary differential equations, iterative approximation, and numerical root solving naturally arise in financial modeling.